In [45]:
%cd /content
!rm -rf Gradu
!git clone https://github.com/Elkkujou/Gradu.git
%cd /content/Gradu
!ls

/content
Cloning into 'Gradu'...
remote: Enumerating objects: 271, done.
remote: Counting objects: 100% (135/135), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 271 (delta 70), reused 0 (delta 0), pack-reused 136 (from 1)
Receiving objects: 100% (271/271), 29.79 MiB | 19.84 MiB/s, done.
Resolving deltas: 100% (128/128), done.
/content/Gradu
 chatti_RF.ipynb	       regime_prediction_famafrench.ipynb  'RF REGIIMI HYVÄ TRAINING.ipynb'
'Financial turbulence.ipynb'   regime_prediction_msci.ipynb	   'RF_regime (3).ipynb'
 FT_source.xlsx		       regime_pred.txt			    THE_ONE.xlsx
 Regiimi_prediction.ipynb      RF_Gradu.ipynb


In [46]:
calc_drawdowns = True
drawdowns_from = 'USA MOMENTUM Standard (Large+Mid Cap)'
cluster_n = 3

normmalize = True
ML_reg = True
ML_long = False
DL = False

In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm
from tabulate import tabulate

from IPython.display import display, HTML
from sklearn.cluster import KMeans

In [48]:
xls_file = pd.ExcelFile("/content/Gradu/THE_ONE.xlsx")
excel_df = xls_file.parse("returns non-log")

excel_df.columns = excel_df.columns.get_level_values(0)
excel_df['Date'] = pd.to_datetime(excel_df['Date'])
excel_df.sort_values('Date', inplace=True)

print("Headers in the 'returns non-log' sheet:")
print(excel_df.columns)

Headers in the 'returns non-log' sheet:
Index(['Date', 'USA Standard (Large+Mid Cap)',
       'USA MOMENTUM Standard (Large+Mid Cap)',
       'USA MINIMUM VOLATILITY (USD) Standard (Large+Mid Cap)',
       'USA RISK WEIGHTED Standard (Large+Mid Cap)',
       'USA SECTOR NEUTRAL QUALITY Standard (Large+Mid Cap)',
       'USA ENHANCED VALUE Standard (Large+Mid Cap)', 'Winning Factor',
       'VIXCLS', 'GDPC1', 'CPI', 'LEI', '10yr', 'Financial Turbulance',
       '(Regime)', 'Financial condition', 'Drawdown', 'HY Spread', 'IG Spread',
       '10y-2y', 'Vix-regimes2', 'Vix-regimes3', 'FT_regimes2', 'FT_regimes3',
       'DD_regimes2', 'DD_regimes3', 'cluster_0 (minimal drawdown',
       'cluster_1 (moderate drawdown)', 'cluster_2 (severe drawdown)', 'SMB',
       'HML', 'RMW', 'CMA', 'MOM', '(Regimes)', 'Regimes'],
      dtype='object')


In [49]:
if calc_drawdowns:
    # Calculate the cumulative portfolio values using excel_df and the specified drawdowns_from column
    portfolio = (1 + excel_df[drawdowns_from]).cumprod()

    # Define a function to compute the maximum drawdown over a rolling window
    def max_dd(s):
        return (s / s.cummax() - 1).min()

    # Calculate the 3-month rolling drawdown using the portfolio series
    dd_series = portfolio.rolling(window=3, min_periods=3).apply(max_dd, raw=False)

    # Create a temporary DataFrame to store dates and the calculated drawdowns
    dd_temp = pd.DataFrame({
        'Date': excel_df['Date'],
        '3M_Max_Drawdown': dd_series
    })

    # Drop rows with missing drawdown data to ensure complete observations
    dd_temp = dd_temp.dropna()

    print("Drawdown calculation (temporary DataFrame):")
    print(dd_temp.head())


Drawdown calculation (temporary DataFrame):
        Date  3M_Max_Drawdown
2 1990-05-31        -0.014564
3 1990-06-29         0.000000
4 1990-07-31         0.000000
5 1990-08-31        -0.086261
6 1990-09-28        -0.128945


In [50]:
# Fit KMeans clustering on the temporary DataFrame's '3M_Max_Drawdown' column
kmeans = KMeans(n_clusters=cluster_n, random_state=42)
dd_temp['DD_cluster'] = kmeans.fit_predict(dd_temp[['3M_Max_Drawdown']])

# Merge the cluster labels from dd_temp into excel_df by matching on the 'Date' column.
# This adds a new column 'DD_cluster' to excel_df.
excel_df = excel_df.merge(dd_temp[['Date', 'DD_cluster']], on='Date', how='left')

print("excel_df with DD_cluster added:")
from IPython.display import display, HTML
display(HTML(excel_df.head().to_html(index=False)))


excel_df with DD_cluster added:


Date,USA Standard (Large+Mid Cap),USA MOMENTUM Standard (Large+Mid Cap),USA MINIMUM VOLATILITY (USD) Standard (Large+Mid Cap),USA RISK WEIGHTED Standard (Large+Mid Cap),USA SECTOR NEUTRAL QUALITY Standard (Large+Mid Cap),USA ENHANCED VALUE Standard (Large+Mid Cap),Winning Factor,VIXCLS,GDPC1,CPI,LEI,10yr,Financial Turbulance,(Regime),Financial condition,Drawdown,HY Spread,IG Spread,10y-2y,Vix-regimes2,Vix-regimes3,FT_regimes2,FT_regimes3,DD_regimes2,DD_regimes3,cluster_0 (minimal drawdown,cluster_1 (moderate drawdown),cluster_2 (severe drawdown),SMB,HML,RMW,CMA,MOM,(Regimes),Regimes,DD_cluster
1990-03-30,0.020702,0.028022,0.012313,0.013039,0.036524,0.013845,USA SECTOR NEUTRAL QUALITY Standard (Large+Mid Cap),21.40,0.003630,0.004687,63.5,8.59,9.383220,Recovery,0.04713,0.000000,NaN,NaN,0.08,High,Low,Normal Turbulence,Low Turbulence,Normal Drawdown,Moderate Drawdown,1,0,0,0.0162,-0.0292,0.0204,-0.0102,0.0180,Normal,Normal,NaN
1990-04-30,-0.023292,-0.014564,-0.031345,-0.042294,-0.007806,-0.032085,USA SECTOR NEUTRAL QUALITY Standard (Large+Mid Cap),18.10,0.003630,0.002333,63.6,8.79,4.756642,Slowdown,-0.13602,-0.023292,NaN,NaN,0.10,High,High,Normal Turbulence,Low Turbulence,Normal Drawdown,Moderate Drawdown,1,0,0,-0.0033,-0.0259,0.0178,-0.0098,0.0239,Normal,Normal,NaN
1990-05-31,0.089444,0.112070,0.079487,0.080355,0.101743,0.081134,USA MOMENTUM Standard (Large+Mid Cap),16.82,0.003630,0.001552,63.3,8.76,14.686352,Slowdown,-0.15632,0.000000,NaN,NaN,0.19,High,High,Normal Turbulence,High Turbulence,Normal Drawdown,Moderate Drawdown,1,0,0,-0.0235,-0.0383,0.0157,-0.0168,0.0303,Normal,Normal,0.0
1990-06-29,-0.006432,0.022340,-0.010553,-0.010138,0.011909,-0.016679,USA MOMENTUM Standard (Large+Mid Cap),18.39,0.000666,0.006197,63.3,8.48,8.020190,Slowdown,0.08472,-0.006432,NaN,NaN,0.45,High,High,Normal Turbulence,Low Turbulence,Normal Drawdown,Moderate Drawdown,1,0,0,0.0135,-0.0193,-0.0103,-0.0039,0.0240,Normal,Normal,0.0
1990-07-31,-0.004276,0.004471,-0.001134,-0.011361,-0.000470,-0.008691,USA MOMENTUM Standard (Large+Mid Cap),28.18,0.000666,0.004619,63.3,8.47,25.637526,Slowdown,0.24339,-0.010680,NaN,NaN,0.79,Normal,Low,High Turbulence,High Turbulence,Normal Drawdown,Moderate Drawdown,1,0,0,-0.0316,-0.0003,-0.0018,0.0321,0.0610,Normal,Normal,0.0


In [51]:
#FEATURES = ['VIXCLS', 'CPI', 'LEI', '10yr', 'Financial condition', '10y-2y', 'Financial Turbulance']

FEATURES = [
    'VIXCLS',
    'GDPC1',
    'CPI',
    'LEI',
    '10yr',
    'Financial Turbulance',
    'Financial condition',
    'HY Spread',
    'IG Spread',
    '10y-2y'
]


TRUE_REGIME = 'DD_cluster'
DATE = 'Date'

selected_columns =  [DATE] + [TRUE_REGIME] +FEATURES


In [52]:
missing = [col for col in selected_columns if col not in excel_df.columns]
if missing:
    raise ValueError(f"Missing columns in df: {missing}")

# Subset and drop rows with empty values
input_df = excel_df[selected_columns]
orig_rows = len(input_df)
input_df = input_df.dropna()
print(f"Original rows: {orig_rows}, Dropped rows: {orig_rows - len(input_df)}")
print(f"First obs: {input_df[DATE].min()}, Last obs: {input_df[DATE].max()}")

# Display as neat HTML table (adjust 'rows_to_show' as needed)
rows_to_show = 10
from IPython.display import display, HTML
display(HTML(input_df.head(rows_to_show).to_html(index=False)))

Original rows: 416, Dropped rows: 81
First obs: 1996-12-31 00:00:00, Last obs: 2024-10-31 00:00:00


Date,DD_cluster,VIXCLS,GDPC1,CPI,LEI,10yr,Financial Turbulance,Financial condition,HY Spread,IG Spread,10y-2y
1996-12-31,1.0,19.47,0.006454,0.002520,77.7,6.30,7.860576,0.00043,3.04,0.61,0.59
1997-01-31,1.0,20.14,0.006454,0.001886,77.8,6.58,14.956959,-0.03500,2.73,0.59,0.47
1997-02-28,0.0,20.17,0.006454,0.001882,78.3,6.42,13.043063,0.05391,2.84,0.57,0.47
1997-03-31,1.0,19.66,0.016652,0.000626,79.3,6.69,10.821978,0.05726,2.94,0.59,0.43
1997-04-30,1.0,19.92,0.016652,0.000626,79.7,6.89,10.938933,0.00925,2.68,0.61,0.45
1997-05-30,0.0,20.19,0.016652,0.000000,79.9,6.71,4.038058,-0.01056,2.67,0.58,0.43
1997-06-30,0.0,20.53,0.012486,0.001876,80.4,6.49,7.517614,-0.02150,2.71,0.56,0.28
1997-07-31,0.0,23.08,0.012486,0.001248,81.1,6.22,14.684361,0.01571,2.59,0.57,0.37
1997-08-29,1.0,23.81,0.012486,0.002494,82.0,6.30,10.252527,0.03394,2.59,0.54,0.32
1997-09-30,1.0,23.87,0.008539,0.002488,82.3,6.21,6.511195,0.10651,2.99,0.67,0.21


In [53]:
from tabulate import tabulate

# Calculate the number of observations for each regime in the data
regime_counts = input_df[TRUE_REGIME].value_counts().sort_index().reset_index()
regime_counts.columns = [TRUE_REGIME, 'Count']

print("Number of observations per regime:")
print(tabulate(regime_counts, headers='keys', tablefmt='psql', showindex=False))


Number of observations per regime:
+--------------+---------+
|   DD_cluster |   Count |
|--------------+---------|
|            0 |     204 |
|            1 |     102 |
|            2 |      29 |
+--------------+---------+


In [54]:
if normmalize == True:
    from sklearn.preprocessing import StandardScaler

    # Instantiate the scaler and apply it in-place to the FEATURES columns
    scaler = StandardScaler()
    input_df[FEATURES] = scaler.fit_transform(input_df[FEATURES])

    print("Normalization applied. First 10 observations:")
    display(HTML(input_df.head(10).to_html(index=False)))


Normalization applied. First 10 observations:


Date,DD_cluster,VIXCLS,GDPC1,CPI,LEI,10yr,Financial Turbulance,Financial condition,HY Spread,IG Spread,10y-2y
1996-12-31,1.0,-0.110862,0.029919,0.158638,-1.496475,1.871825,-0.411545,-0.000050,-0.909269,-1.082181,-0.415568
1997-01-31,1.0,-0.025812,0.029919,-0.058714,-1.487664,2.062500,0.235650,-0.253834,-1.032720,-1.106567,-0.539840
1997-02-28,0.0,-0.022004,0.029919,-0.059929,-1.443610,1.953543,0.061101,0.383025,-0.988915,-1.130954,-0.539840
1997-03-31,1.0,-0.086743,0.867842,-0.489887,-1.355504,2.137408,-0.141463,0.407021,-0.949092,-1.106567,-0.581264
1997-04-30,1.0,-0.053739,0.867842,-0.490021,-1.320261,2.273604,-0.130797,0.063127,-1.052631,-1.082181,-0.560552
1997-05-30,0.0,-0.019465,0.867842,-0.704261,-1.302640,2.151027,-0.760162,-0.078771,-1.056614,-1.118761,-0.581264
1997-06-30,0.0,0.023694,0.525492,-0.061944,-1.258586,2.001211,-0.442824,-0.157134,-1.040684,-1.143147,-0.736604
1997-07-31,0.0,0.347390,0.525492,-0.276851,-1.196911,1.817346,0.210789,0.109400,-1.088472,-1.130954,-0.643400
1997-08-29,1.0,0.440056,0.525492,0.149492,-1.117615,1.871825,-0.193398,0.239981,-1.088472,-1.167534,-0.695180
1997-09-30,1.0,0.447673,0.201170,0.147369,-1.091183,1.810537,-0.534610,0.759796,-0.929180,-1.009021,-0.809096


In [55]:
if ML_reg == True:

    import matplotlib.pyplot as plt
    import seaborn as sns
    from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
    from sklearn.linear_model import LogisticRegression
    from sklearn.svm import SVC
    from sklearn.naive_bayes import GaussianNB
    from sklearn.metrics import accuracy_score, confusion_matrix
    import numpy as np
    import pandas as pd
    from IPython.display import display, HTML
    import itertools

    # Define predictors and target using the given column lists
    predictors = FEATURES
    target = TRUE_REGIME

    # Drop rows with missing values in predictors or target
    data = input_df.dropna(subset=predictors + [target]).reset_index(drop=True)
    X = data[predictors]
    y = data[target]
    # Also grab the DATE column for printing prediction timestamps
    dates = data[DATE]

    # Define the models (including GNB)
    models = {
        'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42),
        'GradientBoosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
        'LogisticRegression': LogisticRegression(max_iter=1000, random_state=42),
        'SVC_linear': SVC(kernel='linear', probability=True, random_state=42),
        'SVC_rbf': SVC(kernel='rbf', probability=True, random_state=42),
        'GNB': GaussianNB()
    }

    window_size = 60  # e.g., 60 observations for training
    unique_regimes = np.unique(y)

    # Dictionary to store ensemble results for each combination subset
    ensemble_results = {}
    model_names = list(models.keys())

    # Loop over all possible subset sizes (pairs, triplets, quartets, quintets, full set)
    for r in range(2, len(model_names) + 1):
        for combo in itertools.combinations(model_names, r):
            combo_name = "+".join(combo)
            combo_acc = []          # accuracy for each rolling iteration
            combo_vote_details = [] # list of vote-count dicts for each iteration
            combo_y_true = []       # true label for each iteration
            combo_y_pred = []       # ensemble prediction for each iteration
            print(f"Evaluating combination: {combo_name}")
            for i in range(len(X)):
                if i < window_size:
                    continue
                # Define training and test sets for the rolling window
                X_train_window = X.iloc[i-window_size:i]
                y_train_window = y.iloc[i-window_size:i]
                X_test_window = X.iloc[[i]]
                y_test_window = y.iloc[[i]]
                preds = []
                # For each model in the combination, fit and predict
                for name in combo:
                    model = models[name]
                    model.fit(X_train_window, y_train_window)
                    pred = model.predict(X_test_window)[0]
                    preds.append(pred)
                # Count votes for each regime in this iteration
                vote_count = {reg: preds.count(reg) for reg in unique_regimes}
                combo_vote_details.append(vote_count)
                # Majority vote for ensemble prediction
                ensemble_prediction = max(set(preds), key=preds.count)
                combo_acc.append(1.0 if ensemble_prediction == y_test_window.values[0] else 0.0)
                combo_y_true.append(y_test_window.values[0])
                combo_y_pred.append(ensemble_prediction)
            # Average ensemble accuracy over all iterations
            avg_acc = np.mean(combo_acc)
            # For each regime, compute average vote count over all iterations
            avg_votes = {reg: np.mean([vote[reg] for vote in combo_vote_details]) for reg in unique_regimes}
            # Convert average vote counts to percentages (relative to ensemble size r)
            avg_vote_percent = {reg: (avg_votes[reg] / r) * 100 for reg in unique_regimes}
            ensemble_results[combo_name] = {
                'average_accuracy': avg_acc,
                'iterations': len(combo_acc),
                'average_votes': avg_votes,
                'average_vote_percent': avg_vote_percent,
                'y_true': combo_y_true,
                'y_pred': combo_y_pred
            }
            print(f"Combination {combo_name} average accuracy: {avg_acc:.4f}")

    # Prepare a summary DataFrame with key information for each combination
    summary_data = []
    for combo_name, result in ensemble_results.items():
        row = {
            'Combination': combo_name,
            'Models': len(combo_name.split('+')),
            'Iterations': result['iterations'],
            'Average Accuracy (%)': f"{result['average_accuracy']*100:.2f}"
        }
        # Add average vote percentage for each regime
        for reg in unique_regimes:
            row[f'Avg Vote % ({reg})'] = f"{result['average_vote_percent'][reg]:.2f}"
        summary_data.append(row)
    df_summary = pd.DataFrame(summary_data)
    print("\nEnsemble Results Summary:")
    display(HTML(df_summary.to_html(index=False)))


Evaluating combination: RandomForest+GradientBoosting
Combination RandomForest+GradientBoosting average accuracy: 0.6545
Evaluating combination: RandomForest+LogisticRegression
Combination RandomForest+LogisticRegression average accuracy: 0.7018
Evaluating combination: RandomForest+SVC_linear
Combination RandomForest+SVC_linear average accuracy: 0.7127
Evaluating combination: RandomForest+SVC_rbf
Combination RandomForest+SVC_rbf average accuracy: 0.6764
Evaluating combination: RandomForest+GNB
Combination RandomForest+GNB average accuracy: 0.6800
Evaluating combination: GradientBoosting+LogisticRegression
Combination GradientBoosting+LogisticRegression average accuracy: 0.6836
Evaluating combination: GradientBoosting+SVC_linear
Combination GradientBoosting+SVC_linear average accuracy: 0.6836
Evaluating combination: GradientBoosting+SVC_rbf
Combination GradientBoosting+SVC_rbf average accuracy: 0.6655
Evaluating combination: GradientBoosting+GNB
Combination GradientBoosting+GNB average 

Combination,Models,Iterations,Average Accuracy (%),Avg Vote % (0.0),Avg Vote % (1.0),Avg Vote % (2.0)
RandomForest+GradientBoosting,2,275,65.45,64.55,28.00,7.45
RandomForest+LogisticRegression,2,275,70.18,68.73,23.64,7.64
RandomForest+SVC_linear,2,275,71.27,69.82,23.09,7.09
RandomForest+SVC_rbf,2,275,67.64,75.45,20.00,4.55
RandomForest+GNB,2,275,68.00,70.36,22.91,6.73
GradientBoosting+LogisticRegression,2,275,68.36,67.09,24.36,8.55
GradientBoosting+SVC_linear,2,275,68.36,68.18,23.82,8.00
GradientBoosting+SVC_rbf,2,275,66.55,73.82,20.73,5.45
GradientBoosting+GNB,2,275,66.55,68.73,23.64,7.64
LogisticRegression+SVC_linear,2,275,69.45,72.36,19.45,8.18


In [56]:
if ML_long == True:

  #Tää Model testaa training windowit 12 -> 120 kk ja näyttää kaikille accuracyt + regiimi kohtaset accuracyt

  import matplotlib.pyplot as plt
  import seaborn as sns
  from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
  from sklearn.linear_model import LogisticRegression
  from sklearn.svm import SVC
  from sklearn.metrics import accuracy_score, confusion_matrix
  import numpy as np
  import pandas as pd
  from IPython.display import display, HTML

  # Define predictors and target using the given column lists
  predictors = FEATURES
  target = TRUE_REGIME

  # Drop rows with missing values in predictors or target
  data = input_df.dropna(subset=predictors + [target]).reset_index(drop=True)
  X = data[predictors]
  y = data[target]
  # Also grab the DATE column for printing prediction timestamps
  dates = data[DATE]

  # Define the models
  models = {
      'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42),
      'GradientBoosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
      'LogisticRegression': LogisticRegression(max_iter=2000, random_state=42),  # Increased max_iter
      'SVC_linear': SVC(kernel='linear', probability=True, random_state=42),
      'SVC_rbf': SVC(kernel='rbf', probability=True, random_state=42)
  }

  # Get the unique regimes from the target
  unique_regimes = np.unique(y)

  # Lists to store overall results (one row per training window size)
  overall_results = []

  # Dictionary to store regime-specific results.
  # Keys are the regime values; values will be a list of dictionaries (one per training window size)
  regime_results = {reg: [] for reg in unique_regimes}

  # Loop over training windows from 1 year (12 months) to 10 years (120 months)
  for window_size in range(12, 121, 12):
      print("="*50)
      print(f"Running rolling window evaluation with training window of {window_size} months")

      # Initialize dictionaries for this run
      rolling_accuracies = {name: [] for name in models}
      pred_true = {name: {'y_true': [], 'y_pred': []} for name in models}

      # Loop over the dataset in a rolling window manner
      for i in range(len(X)):
          if i < window_size:
              continue

          # Training period: previous 'window_size' observations
          train_end = dates.iloc[i-1]  # Last date in training window
          test_date = dates.iloc[i]     # Date for prediction

          X_train_window = X.iloc[i-window_size:i]
          y_train_window = y.iloc[i-window_size:i]
          X_test_window = X.iloc[[i]]
          y_test_window = y.iloc[[i]]

          # Loop through each model, fit, predict, and store accuracies
          for name, model in models.items():
              # Check if the training window contains at least 2 unique classes
              if len(np.unique(y_train_window)) < 2:
                  print(f"Skipping model {name} for timestamp {test_date} due to only one class in training window.")
                  rolling_accuracies[name].append(np.nan)
                  pred_true[name]['y_true'].append(y_test_window.values[0])
                  pred_true[name]['y_pred'].append(None)
                  continue

              model.fit(X_train_window, y_train_window)
              y_pred = model.predict(X_test_window)
              acc = accuracy_score(y_test_window, y_pred)
              rolling_accuracies[name].append(acc)
              pred_true[name]['y_true'].append(y_test_window.values[0])
              pred_true[name]['y_pred'].append(y_pred[0])

      # Compute overall average rolling window accuracy for each model
      # Use np.nanmean to ignore NaN values
      avg_accuracies = {name: np.nanmean(rolling_accuracies[name]) if rolling_accuracies[name] else 0
                        for name in models}

      # Build a result row for overall accuracy
      overall_row = {
          "Training Window (months)": window_size,
          "RandomForest": f"{avg_accuracies['RandomForest']*100:.2f}%" if not np.isnan(avg_accuracies['RandomForest']) else "N/A",
          "GradientBoosting": f"{avg_accuracies['GradientBoosting']*100:.2f}%" if not np.isnan(avg_accuracies['GradientBoosting']) else "N/A",
          "LogisticRegression": f"{avg_accuracies['LogisticRegression']*100:.2f}%" if not np.isnan(avg_accuracies['LogisticRegression']) else "N/A",
          "SVC_linear": f"{avg_accuracies['SVC_linear']*100:.2f}%" if not np.isnan(avg_accuracies['SVC_linear']) else "N/A",
          "SVC_rbf": f"{avg_accuracies['SVC_rbf']*100:.2f}%" if not np.isnan(avg_accuracies['SVC_rbf']) else "N/A"
      }
      overall_results.append(overall_row)

      # For each regime, compute regime-specific accuracy per model from the predictions made in this run
      for reg in unique_regimes:
          regime_acc = {}
          for name in models:
              # Filter predictions for the current regime (using true labels)
              y_true_reg = [yt for yt in pred_true[name]['y_true'] if yt == reg]
              y_pred_reg = [yp for yt, yp in zip(pred_true[name]['y_true'], pred_true[name]['y_pred']) if yt == reg]
              if len(y_true_reg) > 0:
                  acc_reg = np.mean(np.array(y_true_reg) == np.array(y_pred_reg))
              else:
                  acc_reg = np.nan
              regime_acc[name] = acc_reg

          # Build a row for this regime and training window
          regime_row = {"Training Window (months)": window_size}
          for name in models:
              regime_row[name] = f"{regime_acc[name]*100:.2f}%" if not np.isnan(regime_acc[name]) else "N/A"
          regime_results[reg].append(regime_row)

      print(f"Completed training window = {window_size} months")
      print("="*50)

  # Display overall results table
  df_overall = pd.DataFrame(overall_results)
  print("Overall Rolling Window Average Accuracy:")
  display(HTML(df_overall.to_html(index=False)))

  # Now display separate tables for each regime
  for reg in unique_regimes:
      df_regime = pd.DataFrame(regime_results[reg])
      print(f"\nRolling Window Accuracy for Regime {reg}:")
      display(HTML(df_regime.to_html(index=False)))

  # Optionally, you can also compute and display confusion matrices per model if desired.


In [57]:
if DL == True:
    import warnings
    warnings.filterwarnings("ignore")  # Suppress warnings

    import tensorflow as tf
    from tensorflow.keras.models import Sequential, Model
    from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
    import xgboost as xgb
    from sklearn.metrics import confusion_matrix, accuracy_score
    import matplotlib.pyplot as plt
    import seaborn as sns
    import numpy as np
    import pandas as pd
    from IPython.display import display, HTML

    # Define the experiment function with remapping of training labels if needed
    def run_experiment(training_window, seq_len,
                       lstm_units_first=128, lstm_units_second=32, dropout_rate=0.3,
                       xgb_n_estimators=100, xgb_max_depth=4, xgb_learning_rate=0.1):
        """
        Runs the rolling-window hybrid (LSTM + XGBoost) experiment using a given training_window (in months)
        and a sequence length (seq_len). If the training labels do not form a contiguous set starting at 0,
        they are remapped. Returns overall accuracy, total number of predictions, and regime-specific accuracies.
        """
        all_preds = []   # To store predictions
        all_true = []    # To store true regime labels

        # Ensure data is sorted by date
        data = input_df.sort_values(DATE).reset_index(drop=True)

        # Rolling window: iterate from index=training_window to end of data
        for i in range(training_window, len(data)):
            # Use the previous 'training_window' months as training data
            train_window = data.iloc[i - training_window:i].reset_index(drop=True)
            if len(train_window) < training_window:
                continue

            # Generate overlapping sequences of length 'seq_len' within the training window
            X_train_sequences = []
            y_train = []
            for j in range(len(train_window) - seq_len + 1):
                seq = train_window[FEATURES].iloc[j : j + seq_len].values
                target = train_window[TRUE_REGIME].iloc[j + seq_len - 1]  # regime at end of sequence
                X_train_sequences.append(seq)
                y_train.append(target)
            X_train_sequences = np.array(X_train_sequences)  # shape: (num_samples, seq_len, num_features)
            y_train = np.array(y_train)

            # Remap training labels to contiguous integers starting at 0 if needed.
            unique_classes = np.unique(y_train)
            mapping = {old: new for new, old in enumerate(unique_classes)}
            inverse_mapping = {v: k for k, v in mapping.items()}
            y_train_mapped = np.array([mapping[val] for val in y_train])

            # Build the LSTM feature extractor using an explicit Input layer
            input_layer = Input(shape=(seq_len, len(FEATURES)))
            lstm_out = LSTM(lstm_units_first, return_sequences=True)(input_layer)
            lstm_out = LSTM(lstm_units_second, return_sequences=False)(lstm_out)
            dropout_out = Dropout(dropout_rate)(lstm_out)
            feature_extractor = Model(inputs=input_layer, outputs=dropout_out)

            # Extract features from training sequences
            X_train_features = feature_extractor.predict(X_train_sequences, verbose=0)

            # Train an XGBoost classifier on these features using the remapped labels.
            clf = xgb.XGBClassifier(
                n_estimators=xgb_n_estimators,
                max_depth=xgb_max_depth,
                learning_rate=xgb_learning_rate,
                use_label_encoder=False,
                eval_metric='mlogloss'
            )
            clf.fit(X_train_features, y_train_mapped)

            # For prediction, use the last 'seq_len'-month sequence from the training window
            X_pred_seq = train_window[FEATURES].iloc[-seq_len:].values.reshape(1, seq_len, len(FEATURES))
            X_pred_feature = feature_extractor.predict(X_pred_seq, verbose=0)
            pred_mapped = clf.predict(X_pred_feature)[0]
            # Map back to original label
            pred_regime = inverse_mapping[pred_mapped]

            # The true regime for the next month (at time index i)
            true_regime = data[TRUE_REGIME].iloc[i]

            all_preds.append(pred_regime)
            all_true.append(true_regime)

        # Compute overall accuracy and total number of predictions
        overall_acc = accuracy_score(all_true, all_preds)
        total_obs = len(all_preds)

        # Compute regime-specific accuracies
        regimes = np.unique(all_true)
        regime_accuracies = {}
        for r in regimes:
            indices = [idx for idx, val in enumerate(all_true) if val == r]
            if len(indices) > 0:
                correct = sum(1 for idx in indices if all_preds[idx] == all_true[idx])
                regime_acc = correct / len(indices)
                regime_accuracies[r] = regime_acc
            else:
                regime_accuracies[r] = np.nan

        return {
            "training_window": training_window,
            "seq_len": seq_len,
            "total_obs": total_obs,
            "overall_accuracy": overall_acc,
            "regime_accuracies": regime_accuracies
        }

    # Prepare to store experiment results in a list of dictionaries
    results = []

    # Define training windows from 1 to 10 years (in months) – assume 1 year = 12 months
    for tw in range(12, 121, 12):  # 12, 24, ..., 120 months
        # Define sequence length percentages: 25%, 50%, 75%, and 100% of training window
        for frac in [0.25, 0.5, 0.75, 1.0]:
            seq = int(round(tw * frac))
            print("="*50)
            print(f"Running experiment with training_window = {tw} months and seq_len = {seq} months")
            res = run_experiment(training_window=tw, seq_len=seq)
            # Build a result row that includes run parameters and accuracies
            row = {
                "Training Window (months)": tw,
                "Seq Length (months)": seq,
                "Total Observations": res["total_obs"],
                "Overall Accuracy": f"{res['overall_accuracy']*100:.2f}%",
                "Accuracy Regime 0": f"{res['regime_accuracies'].get(0, np.nan)*100:.2f}%" if 0 in res["regime_accuracies"] else "N/A",
                "Accuracy Regime 1": f"{res['regime_accuracies'].get(1, np.nan)*100:.2f}%" if 1 in res["regime_accuracies"] else "N/A",
                "Accuracy Regime 2": f"{res['regime_accuracies'].get(2, np.nan)*100:.2f}%" if 2 in res["regime_accuracies"] else "N/A"
            }
            results.append(row)
            print("="*50)

    # Convert results to a DataFrame and display as an HTML table
    df_results = pd.DataFrame(results)
    display(HTML(df_results.to_html(index=False)))


In [58]:
# Convert results to a DataFrame and display as an HTML table
df_results = pd.DataFrame(results)
display(HTML(df_results.to_html(index=False)))


NameError: name 'results' is not defined